In [55]:
import glia, os, numpy as np
from functools import partial
import sklearn.svm as svm
from sklearn.model_selection import GridSearchCV

In [26]:
grating_speed_fp = "/storage/uw/3brain/200805_acuity_tests/R1_E3_AMES_240min_grating_speeds.npz"
checkerboard_contrast_fp = "/storage/uw/3brain/200805_acuity_tests/R1_E4_AMES_340min_checkerboard_contrast.npz"
grating_contrast_fp = "/storage/uw/3brain/200805_acuity_tests/R1_E5_AMES_430min_grating_contrast.npz"
letters_fp = "/storage/uw/3brain/200805_acuity_tests/R1_E6_AMES_540min_letters.npz"

## get experimental parameters 

In [33]:
px_per_deg = 12.524 # darwin to confirm latest value...
px_to_cpd = partial(glia.px_to_cpd, px_per_deg=px_per_deg)
px_to_logmar = partial(glia.px_to_logmar, px_per_deg=px_per_deg)

In [37]:
data_dir = os.path.split(grating_speed_fp)[0]
lab_notebook = glia.open_lab_notebook(data_dir + "/lab notebook.yml")

name = os.path.split(grating_speed_fp)[1][:-4] # drop .npz
stimulus_file = os.path.join(data_dir, name + ".stim")
metadata, stimulus_list, method = glia.read_stimulus(stimulus_file)
sizes = glia.get_stimulus_parameters(stimulus_list, 'GRATING', "width")
grating_speed_sizes = list(map(px_to_cpd, sizes))
grating_speed_conditions = glia.get_stimulus_parameters(stimulus_list, 'GRATING', 'speed')

name = os.path.split(checkerboard_contrast_fp)[1][:-4] # drop .npz
stimulus_file = os.path.join(data_dir, name + ".stim")
metadata, stimulus_list, method = glia.read_stimulus(stimulus_file)
checkerboard_contrast_sizes = list(map(px_to_cpd, sizes))
checkerboard_contrast_conditions = glia.get_checkerboard_contrasts(stimulus_list)

name = os.path.split(grating_contrast_fp)[1][:-4] # drop .npz
stimulus_file = os.path.join(data_dir, name + ".stim")
metadata, stimulus_list, method = glia.read_stimulus(stimulus_file)
grating_speed_sizes = list(map(px_to_cpd, sizes))
grating_speed_conditions = glia.get_grating_contrasts(stimulus_list)

name = os.path.split(letters_fp)[1][:-4] # drop .npz
stimulus_file = os.path.join(data_dir, name + ".stim")
metadata, stimulus_list, method = glia.read_stimulus(stimulus_file)
letter_sizes = glia.get_stimulus_parameters(stimulus_list, "LETTER", 'size')
letter_sizes = px_to_logmar(letter_sizes)

In [38]:
grating_speed_sizes, grating_speed_conditions

([0.36835294117647055,
  0.2319259259259259,
  0.1456279069767442,
  0.09208823529411764,
  0.0585233644859813,
  0.036835294117647056],
 [0.3990440051725493,
  0.5010548441466505,
  0.6286924914335388,
  0.7957504163913921,
  1.0])

In [39]:
checkerboard_contrast_sizes, checkerboard_contrast_conditions

([0.36835294117647055,
  0.2319259259259259,
  0.1456279069767442,
  0.09208823529411764,
  0.0585233644859813,
  0.036835294117647056],
 [0.31519833051120055,
  0.3990440051725493,
  0.5010548441466505,
  0.6286924914335388,
  0.7957504163913921,
  1.0])

In [40]:
grating_speed_sizes, grating_speed_conditions

([0.36835294117647055,
  0.2319259259259259,
  0.1456279069767442,
  0.09208823529411764,
  0.0585233644859813,
  0.036835294117647056],
 [0.3990440051725493,
  0.5010548441466505,
  0.6286924914335388,
  0.7957504163913921,
  1.0])

In [47]:
letter_sizes

[2.0783482001108036,
 2.379378195774785,
 2.680408191438766,
 2.9814381871027473,
 3.1575294461584282,
 3.2824681827667286,
 3.379378195774785,
 3.4585594418224095]

## letter SVM

In [50]:
data = np.load(letters_fp)

In [51]:
training_100ms = glia.bin_100ms(np.expand_dims(data["training_data"],0))
validation_100ms = glia.bin_100ms(np.expand_dims(data["validation_data"],0))

In [59]:
for i, size in enumerate(letter_sizes):
    print(f'=== SVC for logmar {size} ===')
    # note: no expand dims, hardcoded 1 ncondition
    training_target = data["training_target"][i]
    validation_target = data["validation_target"][i]
    svr = svm.SVC()
    parameters = {'C': [1, 10, 100, 1000],
                  'gamma': [0.001, 0.0001]},
    clf = GridSearchCV(svr, parameters, n_jobs=12)
    report, confusion = glia.classifier_helper(clf,
        (training_100ms[0,i], training_target),
        (validation_100ms[0,i], validation_target))
    print(report)
    print(confusion)

=== SVC for logmar 2.0783482001108036 ===


/home/tyler/lib/anaconda3/envs/glia/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.33      0.10      0.15        10
           2       0.25      0.10      0.14        10
           3       0.00      0.00      0.00        10
           4       0.05      0.10      0.06        10
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           7       0.15      0.50      0.23        10
           8       1.00      0.10      0.18        10
           9       0.25      0.10      0.14        10
          10       0.10      0.40      0.16        10

    accuracy                           0.13       110
   macro avg       0.19      0.13      0.10       110
weighted avg       0.19      0.13      0.10       110

       BLANK  C  D  H  K  N  O  R  S  V  Z
BLANK      0  0  0  0  4  0  0  4  0  0  2
C          0  1  0  0  0  0  1  3  0  0  5
D          0  0  1  0  2  0  0  4  0  0  3
H          0  0